# Pajarito Solver

See https://github.com/JuliaOpt/Pajarito.jl

Pajarito is a mixed-integer convex programming (MICP) solver package written in Julia.

MICP problems are convex except for restrictions that some variables take binary or integer values. Pajarito supports both mixed-integer conic programming, which encodes nonlinearities using a small number of predefined cones, and more traditional convex mixed-integer nonlinear programming, which encodes nonlinearities with smooth functions and uses their derivatives.

Pajarito solves MICP problems by constructing sequential lifted polyhedral outer-approximations of the convex feasible set to leverage the power of MILP solvers. The algorithm has theoretical finite-time convergence under reasonable assumptions. Pajarito accesses state-of-the-art MILP solvers and continuous conic or nonlinear programming (NLP) solvers through the MathProgBase interface.



## Optimal experimental design

Featured in Pajarito examples folder at https://github.com/JuliaOpt/Pajarito.jl/blob/master/examples/expdesign.jl

Experimental design examples (D-optimal, A-optimal, and E-optimal) from Boyd and Vandenberghe, "Convex Optimization", section 7.5 (CVX code adapted from expdesign.m by Lieven Vandenberghe and Almir Mutapcic)

`D-optimal design' minimizes the volume of a confidence ellipsoid defined by the error covariance (Fisher matrix):

$$ 
\max \quad \log \det \sum_i \lambda_i V_i V_i^T
\\
s.t. \quad \sum_i \lambda_i = 1,\quad \lambda_i \geq 0, \forall i
$$


### Install packages

In [ ]:
Pkg.update()
Pkg.add("Convex")
Pkg.add("Pajarito")
Pkg.checkout("Pajarito")
Pkg.add("Cbc")
Pkg.checkout("Cbc")
Pkg.add("SCS")

### Specify MICP solver

In [ ]:
using Cbc
mip_solver = CbcSolver(logLevel=0)

using SCS
cont_solver = SCSSolver(eps=1e-6, max_iters=100000, verbose=0)

using Pajarito
solver = PajaritoSolver(
    mip_solver_drives=false,
    log_level=3,
    rel_gap=1e-4,
    mip_solver=mip_solver,
    cont_solver=cont_solver,
)

### Specify/generate data

In [ ]:
# Uncomment only the line with the desired data dimensions
(q, p, n, nmax) = (
    # 100, 250, 500, 5   # Huge
    # 25, 75, 125, 5     # Large
    # 10, 30, 50, 5      # Medium
    # 5, 15, 25, 5       # Small
    4, 8, 12, 3        # Tiny
)
@assert (p > q) && (n > q)

# Generate matrix of experimental vectors
# Change or comment random seed to get different random V matrix
srand(100)

V = Array{Float64}(q, p)
for i in 1:q, j in 1:p
    v = randn()
    if abs(v) < 1e-2
        v = 0.
    end
    V[i, j] = v
end

### Model with Convex.jl and solve

In [ ]:
using Convex

λ = Convex.Variable(p, :Int)
Q = Convex.Variable(q, q)

dOpt = maximize(
    logdet(Q),
    Q == V * diagm(λ./n) * V',
    sum(λ) <= n,
    λ >= 0,
    λ <= nmax
)

solve!(dOpt, solver)
println("\n  objective $(dOpt.optval)")
println("  solution\n$(λ.value)")

## Sum of squares trajectory planning

Credit to Joey Huchette for Julia code, and Robin Deits and Prof. Russ Tedrake (MIT) and collaborators for the quadrotor application (see http://groups.csail.mit.edu/robotics-center/public_papers/Landry15b.pdf)

A short video provides an overview: https://www.youtube.com/watch?v=7epTZM0yHA4

For simplicity, we just use boxes for the convex segments, but one could use arbitrary semialgebraic (described by finite number of polynomials) sets

### Install packages

In [ ]:
Pkg.update()
Pkg.add("Pajarito")
Pkg.add("Cbc")
Pkg.add("CSDP")
Pkg.checkout("CSDP")
Pkg.add("JuMP")
Pkg.checkout("JuMP")
Pkg.add("PolyJuMP")
Pkg.checkout("PolyJuMP")
Pkg.add("SumOfSquares")
Pkg.checkout("SumOfSquares")
Pkg.add("MultivariatePolynomials")
Pkg.checkout("MultivariatePolynomials")

### Specify MICP solver

In [ ]:
using Cbc
mip_solver = CbcSolver(logLevel=0)

using CSDP
cont_solver = CSDPSolver(printlevel=0,fastmode=1)

using Pajarito
solver = PajaritoSolver(
    mip_solver_drives=false,
    log_level=3,
    rel_gap=1e-4,
    mip_solver=mip_solver,
    cont_solver=cont_solver,
    prim_cuts_always=true,
)

### Specify types and data

In [ ]:
type Box
    xl::Float64
    xu::Float64
    yl::Float64
    yu::Float64
end

boxes = Box[Box(0.0,1.0,0.0,0.3),
            Box(0.8,1.7,0.1,0.3),
            Box(1.4,1.9,0.2,0.4),
            Box(1.0,1.7,0.3,0.5),
            Box(0.5,1.4,0.4,0.6),
            Box(0.0,1.0,0.5,0.7),
            Box(0.2,1.0,0.6,0.8),
            Box(0.5,1.3,0.7,0.9),
            Box(1.0,2.0,0.7,1.0)]

N = 8 # Number of trajectory pieces
d = 2 # dimension of space
r = 5 # dimension of polynomial trajectories
M = 2 # number of horizontal segments

domain = Box(0,M,0,1)

X₀   = Dict(:x=>0, :y=>0)
X₀′  = Dict(:x=>1, :y=>0)
X₀′′ = Dict(:x=>0, :y=>0)
X₁   = Dict(:x=>2, :y=>1)
X₁′  = Dict(:x=>1, :y=>0)
X₁′′ = Dict(:x=>0, :y=>0)

# T = linspace(1, 2, N+1)
T = linspace(0, 1, N+1)

Tmin = minimum(T)
Tmax = maximum(T)
;

### Model with JuMP and polynomial packages

In [ ]:
using JuMP, PolyJuMP, SumOfSquares, MultivariatePolynomials

model = SOSModel(solver=solver)

@polyvar(t)
Z = monomials([t], 0:r)

@variable(model, H[1:N,boxes], Bin)

Mxl, Mxu, Myl, Myu = domain.xl, domain.xu, domain.yl, domain.yu
p = Dict()
for j in 1:N
    @constraint(model, sum(H[j,box] for box in boxes) == 1)

    p[(:x,j)] = @polyvariable(model, _, Z)
    p[(:y,j)] = @polyvariable(model, _, Z)
    for box in boxes
        xl, xu, yl, yu = box.xl, box.xu, box.yl, box.yu
        @assert xl >= Mxl
        @polyconstraint(model, p[(:x,j)] >= Mxl + (xl-Mxl)*H[j,box], domain = (t >= T[j] && t <= T[j+1]))
        @assert xu <= Mxu
        @polyconstraint(model, p[(:x,j)] <= Mxu + (xu-Mxu)*H[j,box], domain = (t >= T[j] && t <= T[j+1]))
        @assert yl >= Myl
        @polyconstraint(model, p[(:y,j)] >= Myl + (yl-Myl)*H[j,box], domain = (t >= T[j] && t <= T[j+1]))
        @assert yu <= Myu
        @polyconstraint(model, p[(:y,j)] <= Myu + (yu-Myu)*H[j,box], domain = (t >= T[j] && t <= T[j+1]))
    end
end

for axis in (:x,:y)
    @constraint(model,               p[(axis,1)       ]([Tmin], [t]) == X₀[axis])
    @constraint(model, differentiate(p[(axis,1)], t   )([Tmin], [t]) == X₀′[axis])
    @constraint(model, differentiate(p[(axis,1)], t, 2)([Tmin], [t]) == X₀′′[axis])

    for j in 1:N-1
        # @constraint(model,               T[j+1]^(-d)*(p[(axis,j)       ]([T[j+1]], [t])) ==               T[j+1]^(-d)*(p[(axis,j+1)       ]([T[j+1]], [t])))
        # @constraint(model, T[j+1]^(-d)*(differentiate(p[(axis,j)], t   )([T[j+1]], [t])) == T[j+1]^(-d)*(differentiate(p[(axis,j+1)], t   )([T[j+1]], [t])))
        # @constraint(model, T[j+1]^(-d)*(differentiate(p[(axis,j)], t, 2)([T[j+1]], [t])) == T[j+1]^(-d)*(differentiate(p[(axis,j+1)], t, 2)([T[j+1]], [t])))
        @constraint(model,               p[(axis,j)       ]([T[j+1]], [t]) ==               p[(axis,j+1)       ]([T[j+1]], [t]))
        @constraint(model, differentiate(p[(axis,j)], t   )([T[j+1]], [t]) == differentiate(p[(axis,j+1)], t   )([T[j+1]], [t]))
        @constraint(model, differentiate(p[(axis,j)], t, 2)([T[j+1]], [t]) == differentiate(p[(axis,j+1)], t, 2)([T[j+1]], [t]))
    end

    @constraint(model,               p[(axis,N)       ]([Tmax], [t]) == X₁[axis])
    # @constraint(model, differentiate(p[(axis,N)], t   )([1], [t]) == X₁′[axis])
    # @constraint(model, differentiate(p[(axis,N)], t, 2)([1], [t]) == X₁′′[axis])
end

@variable(model, γ[keys(p)] ≥ 0)
for (key,val) in p
    @constraint(model, γ[key] ≥ norm(differentiate(val, t, 3)))
end
# expr = norm(differentiate(poly, t, 3) for poly in values(p))
# @constraint(model, γ ≥ sum(norm(differentiate(poly, t, 3)) for poly in values(p)))
@objective(model, Min, sum(γ))
;

### Solve and interpret solution

In [ ]:
solve(model)

PP = Dict(key => getvalue(p[key]) for key in keys(p))
HH = getvalue(H)

function eval_poly(r)
    for i in 1:N
        if T[i] <= r <= T[i+1]
            return PP[(:x,i)]([r], [t]), PP[(:y,i)]([r], [t])
            break
        end
    end
    error("Time $r out of interval [$(minimum(T)),$(maximum(T))]")
end

### Plot solution with SFML

See https://github.com/zyedidia/SFML.jl

"This is a binding of the C++ game and multimedia library SFML (Simple and Fast Multimedia Library), developed by Laurent Gomila, for Julia. SFML is often used for game development but it can be used for anything graphics-related"

In [ ]:
Pkg.add("SFML")

In [ ]:
using SFML

const window_width = 800
const window_height = 600

window = RenderWindow("Helicopter", window_width, window_height)
event = Event()

rects = RectangleShape[]
for box in boxes
    rect = RectangleShape()
    xl = (window_width/M)*box.xl
    xu = (window_width/M)*box.xu
    yl = window_height*(domain.yu-box.yl)
    yu = window_height*(domain.yu-box.yu)
    set_size(rect, Vector2f(xu-xl, yu-yl))
    set_position(rect, Vector2f(xl, yl))
    set_fillcolor(rect, SFML.white)
    push!(rects, rect)
end

type Helicopter
    shape::CircleShape
    past_path::Vector{Vector2f}
    path_func::Function
end

const radius = 10

heli = Helicopter(CircleShape(), Vector2f[Vector2f(X₀[:x]*window_width, X₀[:y]*window_height)], eval_poly)
set_position(heli.shape, Vector2f(window_width/2,window_height/2))
set_radius(heli.shape, radius)
set_fillcolor(heli.shape, SFML.red)
set_origin(heli.shape, Vector2f(radius, radius))

function update_heli!(heli::Helicopter, tm)
    (_x,_y) = heli.path_func(tm)
    x = window_width / M * _x
    y = window_height * (1-_y)
    pt = Vector2f(x,y)
    set_position(heli.shape, pt)
    # move(heli.shape, pt-heli.past_path[end])
    push!(heli.past_path, pt)
    get_position(heli.shape)
    nothing
end

const maxtime = 10.0

make_gif(window, window_width, window_height, 1.05*maxtime, "foobarbat.gif", 0.05)

clock = Clock()
restart(clock)

while isopen(window)
    frametime = as_seconds(get_elapsed_time(clock))

    @show normalizedtime = Tmin + (frametime / maxtime)*(Tmax-Tmin)

    (normalizedtime >= Tmax) && break

    while pollevent(window, event)
        if get_type(event) == EventType.CLOSED
            close(window)
        end
    end

    clear(window, SFML.blue)

    for rect in rects
        draw(window, rect)
    end
    update_heli!(heli, normalizedtime)
    draw(window, heli.shape)
    display(window)
end